In [2]:
a = {'x':1,'d':6,'b':4,'c':3,'a':2}
ks = [ky for ky in a.keys() if a[ky] > 3 and isinstance(a[ky],int)]
for k in ks:
    a.pop(k)
display(a)
b = []
print(b[-1])

{'x': 1, 'c': 3, 'a': 2}

IndexError: list index out of range

In [12]:
import numpy as np
import torch
from transformers import AutoTokenizer
from verl.utils.dataset.rl_dataset import RLHFDataset, collate_fn
from torch.utils.data.sampler import RandomSampler,BatchSampler,Sampler

from torchdata.stateful_dataloader import StatefulDataLoader

RSN_PATH="/home/azureuser/cloudfiles/code/SRC/O1/reasoningimprove"
train_files = f"{RSN_PATH}/domain/Med/data/tmpds/VerifiableQAllRnd1K_vrl/train.parquet"

batch_size=16
drop_last=True

qwnR1B7Tknz = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B")

train_dataset = RLHFDataset(parquet_files=train_files,
                                         tokenizer=qwnR1B7Tknz,
                                         processor=None,
                                         prompt_key='prompt',
                                         image_key='images',
                                         max_prompt_length=1024, #self.config.data.max_prompt_length,
                                         return_raw_chat=False, #self.config.data.get('return_raw_chat', False),
                                         truncation='error', #self.config.data.get('truncation', 'error'),
                                         filter_overlong_prompts=False, #self.config.data.filter_overlong_prompts,
                                         num_workers=None, #self.config.data.get('filter_overlong_prompts_workers', None)
                                         )

train_dataloader_generator = torch.Generator()
train_dataloader_generator.manual_seed(1024)
sampler = RandomSampler(data_source=train_dataset, generator=train_dataloader_generator)
batch_sampler = BatchSampler(sampler, batch_size, drop_last)

dataset len: 1000


In [13]:
train_dataset[888]

{'data_source': 'vqVEsy1',
 'ability': 'med',
 'reward_model': {'ground_truth': '358.9 mOsm/L', 'style': 'rule'},
 'extra_info': {'index': 888, 'split': 'train'},
 'input_ids': tensor([151643, 151643, 151643,  ..., 151644,  77091,    198]),
 'attention_mask': tensor([0, 0, 0,  ..., 1, 1, 1]),
 'position_ids': tensor([  0,   0,   0,  ..., 191, 192, 193]),
 'raw_prompt_ids': [151644,
  8948,
  198,
  2610,
  525,
  458,
  7485,
  304,
  279,
  2070,
  315,
  15712,
  323,
  33358,
  13,
  3197,
  498,
  4226,
  264,
  3405,
  11,
  498,
  1265,
  1156,
  1744,
  911,
  279,
  32711,
  1882,
  304,
  697,
  3971,
  323,
  1221,
  3410,
  279,
  4226,
  13,
  576,
  32711,
  1882,
  1265,
  387,
  5230,
  304,
  366,
  26865,
  29,
  690,
  26865,
  8066,
  323,
  279,
  4226,
  1265,
  387,
  7407,
  1283,
  279,
  1537,
  690,
  26865,
  8066,
  600,
  1734,
  2572,
  366,
  26865,
  29,
  32711,
  1882,
  1588,
  690,
  26865,
  29,
  4226,
  1588,
  13,
  151645,
  198,
  151644,
  872

In [14]:
_sampler_iter = iter(batch_sampler)

In [15]:
idxLs = next(_sampler_iter)
cnt = 0
kset = set()
while  idxLs:
    for i in idxLs:
        dta = train_dataset[i]
        assert i == dta['extra_info']['index'] and i == dta['index']
        if i not in kset:
            kset.add(i)
        else:
            print(f"**** WE HAVE GOT {i}")
    cnt = cnt + 1
    print(f">>>{cnt}")
    idxLs = next(_sampler_iter)


>>>1
>>>2
>>>3
>>>4
>>>5
>>>6
>>>7
>>>8
>>>9
>>>10
>>>11
>>>12
>>>13
>>>14
>>>15
>>>16
>>>17
>>>18
>>>19
>>>20
>>>21
>>>22
>>>23
>>>24
>>>25
>>>26
>>>27
>>>28
>>>29
>>>30
>>>31
>>>32
>>>33
>>>34
>>>35
>>>36
>>>37
>>>38
>>>39
>>>40
>>>41
>>>42
>>>43
>>>44
>>>45
>>>46
>>>47
>>>48
>>>49
>>>50
>>>51
>>>52
>>>53
>>>54
>>>55
>>>56
>>>57
>>>58
>>>59
>>>60
>>>61
>>>62


StopIteration: 

In [16]:
len(kset)

992

In [24]:
from typing import (
    Generic,
    Iterable,
    Iterator,
    List,
    Optional,
    Sequence,
    Sized,
    TypeVar,
    Union,
)


class BtBatchSampler(Sampler[List[int]]):
    def __init__(
        self,
        sampler: Union[Sampler[int], Iterable[int]],
        batch_size: int,
        drop_last: bool,
        _stdZroSmpDicLs,
        _stdZroSmpThrshld,
    ) -> None:
        # Since collections.abc.Iterable does not check for `__getitem__`, which
        # is one way for an object to be an iterable, we don't do an `isinstance`
        # check here.
        if (
            not isinstance(batch_size, int)
            or isinstance(batch_size, bool)
            or batch_size <= 0
        ):
            raise ValueError(
                f"batch_size should be a positive integer value, but got batch_size={batch_size}"
            )
        if not isinstance(drop_last, bool):
            raise ValueError(
                f"drop_last should be a boolean value, but got drop_last={drop_last}"
            )
        self.sampler = sampler
        self.batch_size = batch_size
        self.drop_last = drop_last
        self._stdZroSmpDicLs = _stdZroSmpDicLs
        self._stdZroSmpThrshld = _stdZroSmpThrshld

    def __iter__(self) -> Iterator[List[int]]:
        batch = [0] * self.batch_size
        idx_in_batch = 0
        found = False
        for idx in self.sampler:
            if idx in self._stdZroSmpDicLs.keys() and len(self._stdZroSmpDicLs[idx]) > self._stdZroSmpThrshld:
                print(f"**FOUND**{idx}")
                found = True
                continue
            batch[idx_in_batch] = idx
            idx_in_batch += 1
            if idx_in_batch == self.batch_size:
                if found:
                    print(f">>>>{batch}<<<<")
                    found = False
                yield batch
                idx_in_batch = 0
                batch = [0] * self.batch_size
        if (not self.drop_last) and idx_in_batch > 0:
            yield batch[:idx_in_batch]

    def __len__(self) -> int:
        if self.drop_last:
            return len(self.sampler) // self.batch_size  # type: ignore[arg-type]
        else:
            return (len(self.sampler) + self.batch_size - 1) // self.batch_size  # type: ignore[arg-type]

In [25]:
stdZroSmpDicLs = {
  # 543:[1.1,2.2,3.3],
  # 1001:[1.1,2.2,3.3,4.4],
  # 1003:[1.1,2.2,3.3,4.4],
  # 1005:[1.1,2.2,3.3,4.4],
  # 1007:[1.1,2.2,3.3,4.4],
}
stdZroSmpThrshld=3

Bt_batch_sampler = BtBatchSampler(sampler, batch_size, True, stdZroSmpDicLs, stdZroSmpThrshld)

In [ ]:
bt_sampler_iter = iter(Bt_batch_sampler)
idxLs = next(bt_sampler_iter)
cnt = 0
kset2 = set()
while  idxLs:
    for i in idxLs:
        dta = train_dataset[i]
        assert i == dta['extra_info']['index'] and i == dta['index']
        if i not in kset2:
            kset2.add(i)
        else:
            print(f"**** WE HAVE GOT {i}")
    cnt = cnt + 1
    print(f">>>{cnt}>>>{len(idxLs)}")
    idxLs = next(bt_sampler_iter)

>>>0>>>16
>>>1>>>16
>>>2>>>16
>>>3>>>16
>>>4>>>16
>>>5>>>16
>>>6>>>16
>>>7>>>16
>>>8>>>16
>>>9>>>16
>>>10>>>16
>>>11>>>16
>>>12>>>16
>>>13>>>16
>>>14>>>16
>>>15>>>16
>>>16>>>16
>>>17>>>16
>>>18>>>16
>>>19>>>16
>>>20>>>16
>>>21>>>16
>>>22>>>16
>>>23>>>16
>>>24>>>16
>>>25>>>16
>>>26>>>16
>>>27>>>16
>>>28>>>16
>>>29>>>16
>>>30>>>16
>>>31>>>16
>>>32>>>16
>>>33>>>16
>>>34>>>16
>>>35>>>16
>>>36>>>16
>>>37>>>16
>>>38>>>16
>>>39>>>16
>>>40>>>16
>>>41>>>16
>>>42>>>16
>>>43>>>16
>>>44>>>16
>>>45>>>16
>>>46>>>16
>>>47>>>16
>>>48>>>16
>>>49>>>16
>>>50>>>16
>>>51>>>16
>>>52>>>16
>>>53>>>16
>>>54>>>16
>>>55>>>16
>>>56>>>16
>>>57>>>16
>>>58>>>16
>>>59>>>16
>>>60>>>16
>>>61>>>16


StopIteration: 

In [20]:
len(kset2)

0